In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./cluster_1.csv')

In [3]:
# Location Frequency, Activity, Main Activity Time 지표에 대한 가중치 설정
weights = [0.4, 0.3, 0.3]

In [4]:
# Rating 계산 함수
def calculate_rating(row):
    # 각 지표에 대한 가중치 적용
    weighted_scores = row[['Location Frequency', 'Activity', 'Main Activity Time']] * weights
    # 가중치 적용된 지표들의 합 계산
    total_weighted_score = weighted_scores.sum()
    # Rating 계산 (최대값 5로 제한)
    rating = min(total_weighted_score * 5, 5)
    return rating

In [5]:
# 데이터에 Rating 열 추가
data['Rating'] = data.apply(calculate_rating, axis=1)

In [6]:
# 데이터 저장
data.to_csv('data_with_rating.csv', index=False)

협업 필터링 적용

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [21]:
data.to_csv('data_with_rating.csv', index=False)

In [22]:
# train set, test set 분리
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [23]:
# train set으로 모델 학습
user_venue_matrix = pd.pivot_table(train_data, values='Rating', index='User ID', columns='Venue ID', fill_value=0)
user_similarity_matrix = np.corrcoef(user_venue_matrix)

In [24]:
# 추천 대상 유저 설정 (여기서는 1번 유저)
target_user_index = 0

In [25]:
# 추천 대상 유저와 유사한 다른 유저 인덱스 구하기
similar_user_indexes = np.argsort(user_similarity_matrix[target_user_index])[::-1][1:]


In [26]:
# 추천 대상 유저와 유사한 다른 유저들의 장소 평점 평균 구하기
similar_user_venue_mean = np.mean(user_venue_matrix.iloc[similar_user_indexes], axis=0)


In [27]:
# 추천 대상 유저가 아직 방문하지 않은 장소 추출
unseen_venues = user_venue_matrix.iloc[target_user_index][user_venue_matrix.iloc[target_user_index] == 0].index


In [28]:
# 추천 대상 유저가 아직 방문하지 않은 장소들 중, 유사한 다른 유저들이 높게 평가한 장소 추천
recommendation = similar_user_venue_mean[unseen_venues].sort_values(ascending=False)[:3]


In [29]:
# test set에서 실제값 가져오기
actual_ratings = test_data[test_data['User ID'] == target_user_index]['Rating'].tolist()


In [30]:
# 추천 결과와 실제값 비교하여 성능평가
tp = len(set(recommendation).intersection(set(actual_ratings)))
fp = len(set(recommendation)) - tp
fn = len(set(actual_ratings)) - tp

if tp + fp == 0:
    precision = 0
else:
    precision = tp / (tp + fp)

if tp + fn == 0:
    recall = 0
else:
    recall = tp / (tp + fn)

if precision + recall == 0:
    f1 = 0
else:
    f1 = 2 * (precision * recall) / (precision + recall)


In [31]:
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.0
Recall: 0
F1-score: 0


^^


In [20]:
import pandas as pd
